# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

What was the objective of the study?


what oxycodone exposure and subsequent
withdrawal in the presence or absence of chronic neuropathic pain changes the transcriptional level.

What do the conditions mean?

oxy: Oxycodon


sal: Saline

What do the genotypes mean?

SNI:(Spared Nerve Injury) Nerves are damaged. SNI surgery results in long-lasting mechanical hypersensitivity


Sham: Sham Surgery, have surgery but no nerve damage, providing a baseline to rule out surgical effects on behavior and pain.

Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.

What would you do?
COmpare the different groups with a ttest. or all with a ANOVA. adjust for multiple testing.


Which groups would you compare to each other?

there are 4 groups SNI-Ocy, SNI-sal, SHAM-Oxy, SHAM-Sal.
Here the following pairings make sense. Compare SNI-oxy vs SNI-sal and SHAM-Ocy vs SHAM-sal to compare the medication. Any difference in pain relieve there?
Next we can compare the phenotypes within one class of medication. SNI-oxy vs SHAM-oxy and Sham-sal vs SNI-sal.

Please also mention which outcome you would expect to see from each comparison.

We would expect that oxycodon has a higher pain relieve for all cases over the sal. The SNI group should profit most of the treatment compared to the SHAM group.

Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition?
2. How many samples do you have per genotype?
3. How often do you have each condition per genotype?

In [5]:
import pandas as pd

df= pd.read_excel("/home/mitch/Dokumente/Worklflow/computational-workflows-2025/notebooks/day_02/conditions_runs_oxy_project.xlsx")

In [8]:
df.head(n=100)

,Patient,Run,RNA-seq,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham
0,?,SRR23195505,x,NaN,x,NaN,x,NaN
1,?,SRR23195506,x,NaN,NaN,x,NaN,x
2,?,SRR23195507,x,NaN,x,NaN,NaN,x
3,?,SRR23195508,x,NaN,NaN,x,x,NaN
4,?,SRR23195509,x,NaN,NaN,x,x,NaN
5,?,SRR23195510,x,NaN,x,NaN,x,NaN
6,?,SRR23195511,x,NaN,NaN,x,NaN,x
7,?,SRR23195512,x,NaN,x,NaN,NaN,x
8,?,SRR23195513,x,NaN,x,NaN,x,NaN
9,?,SRR23195514,x,NaN,NaN,x,NaN,x


In [20]:
#combine "condition: Sal" and 'Condition: Oxy' where x ==1 otherwise 0

df_better = df[['Run', 'Condition: Oxy']].copy()
df_better['Condition: Sal'] = df['condition: Sal'].apply(lambda x: 1 if x == 'x' else 0)
df_better['Condition: Oxy'] = df_better['Condition: Oxy'].apply(lambda x: 1 if x == 'x' else 0)

df_better['Genotype: SNI'] = df['Genotype: SNI'].apply(lambda x: 1 if x == 'x' else 0)
df_better['Genotype: Sham'] = df['Genotype: Sham'].apply(lambda x: 1 if x == 'x' else 0)


In [23]:
df_better
print(f'Number of Oxy Condition: {df_better["Condition: Oxy"].sum()}')
print(f'Number of Sal Condition: {df_better["Condition: Sal"].sum()}')
print(f'Number of SNI Genotype: {df_better["Genotype: SNI"].sum()}')
print(f'Number of Sham Genotype: {df_better["Genotype: Sham"].sum()}')
print(f'Number of Sal + SNI: {((df_better["Condition: Sal"] == 1) & (df_better["Genotype: SNI"] == 1)).sum()}')
print(f'Number of Sal + Sham: {((df_better["Condition: Sal"] == 1) & (df_better["Genotype: Sham"] == 1)).sum()}')
print(f'Number of Oxy + SNI: {((df_better["Condition: Oxy"] == 1) & (df_better["Genotype: SNI"] == 1)).sum()}')
print(f'Number of Oxy + Sham: {((df_better["Condition: Oxy"] == 1) & (df_better["Genotype: Sham"] == 1)).sum()}')

Number of Oxy Condition: 8
Number of Sal Condition: 8
Number of SNI Genotype: 8
Number of Sham Genotype: 8
Number of Sal + SNI: 4
Number of Sal + Sham: 4
Number of Oxy + SNI: 4
Number of Oxy + Sham: 4


They were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [24]:
base_count = pd.read_csv("/home/mitch/Dokumente/Worklflow/computational-workflows-2025/notebooks/day_02/base_counts.csv")

In [25]:
base_count

,Run,Bases
0,SRR23195505,6922564500
1,SRR23195506,7859530800
2,SRR23195507,8063298900
3,SRR23195508,6927786900
4,SRR23195509,7003550100
5,SRR23195510,7377388500
6,SRR23195511,6456390900
7,SRR23195512,7462857900
8,SRR23195513,8099181600
9,SRR23195514,7226808600


In [26]:
#map Bases to Run and add to df_better
df_final= pd.merge(df_better, base_count, left_on='Run', right_on='Run')


In [27]:
df_final

,Run,Condition: Oxy,Condition: Sal,Genotype: SNI,Genotype: Sham,Bases
0,SRR23195505,0,1,1,0,6922564500
1,SRR23195506,1,0,0,1,7859530800
2,SRR23195507,0,1,0,1,8063298900
3,SRR23195508,1,0,1,0,6927786900
4,SRR23195509,1,0,1,0,7003550100
5,SRR23195510,0,1,1,0,7377388500
6,SRR23195511,1,0,0,1,6456390900
7,SRR23195512,0,1,0,1,7462857900
8,SRR23195513,0,1,1,0,8099181600
9,SRR23195514,1,0,0,1,7226808600


In [28]:
#the 2 smallest run _id based of bases
df_final.nsmallest(2, 'Bases')

,Run,Condition: Oxy,Condition: Sal,Genotype: SNI,Genotype: Sham,Bases
11,SRR23195516,1,0,1,0,6203117700
6,SRR23195511,1,0,0,1,6456390900


In [30]:
with open('srr_ids.txt', 'w') as f:
    f.write('SRR23195516\n')
    f.write('SRR23195511\n')

print("SRR IDs file created!")


SRR IDs file created!


While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>
When you are done with this shout, so we can discuss the different ideas.